# T-GCN GSL

**Graph Structure Learning for Traffic Prediction**

This repository provides the implementation of the paper "Graph Structure Learning for Traffic Prediction" by Mahmood Amintoosi.

The adjacency matrix should be estimated using GSL. We have already computed them (for different datasets and different prediction lengths) and saved them in the `data` folder. If the estimated matrices are deleted, they will be computed automatically before prediction by the proposed method.

In [1]:
import os
if 'google.colab' in str(get_ipython()) and not os.path.exists('/content/TGCN-PyTorch'):
    !git clone https://github.com/mamintoosi-papers-codes/TGCN-PyTorch.git
    !pip install -q torchmetrics
    %cd TGCN-PyTorch

In [2]:
%%time

datasets = ['sz', 'los'] #  sz=shenzhen, los=losloop
pred_list = [1, 2, 3, 4]

for  dataset in datasets:
    for pre_len in pred_list:
        %run main.py --config configs/tgcn-{dataset}-pre_len{pre_len}.yaml
        %run main.py --config configs/tgcn-{dataset}-gsl-pre_len{pre_len}.yaml
        %run main.py --config configs/tgcn-{dataset}-gsl-adj-pre_len{pre_len}.yaml

c:\Users\m.amintoosi\.conda\envs\pth-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-03-06 14:20:29,993 INFO]Loaded config from configs/tgcn-sz-pre_len1.yaml: {'fit': {'trainer': {'max_epochs': 50, 'accelerator': 'cuda', 'devices': 1}, 'data': {'dataset_name': 'shenzhen', 'batch_size': 64, 'seq_len': 12, 'pre_len': 1}, 'model': {'model': {'class_path': 'models.TGCN', 'init_args': {'hidden_dim': 100, 'use_gsl': 0}}, 'learning_rate': 0.001, 'weight_decay': 0, 'loss': 'mse_with_regularizer'}}}
[2025-03-06 14:20:30,229 INFO]Using device: cuda


TGCN


[2025-03-06 14:20:32,228 INFO]Starting training for 50 epochs...
[2025-03-06 14:20:33,939 INFO][Epoch 1/50] Train Loss: 109.665068, Val Loss: 1853920.250000, RMSE: 6.385045, MAE: 5.018596, Accuracy: 0.5551, R2: 0.6289, Expl.Var: 0.6332
[2025-03-06 14:20:47,919 INFO][Epoch 11/50] Train Loss: 36.290581, Val Loss: 1486394.125000, RMSE: 5.717226, MAE: 4.225228, Accuracy: 0.6016, R2: 0.7005, Expl.Var: 0.7007
[2025-03-06 14:21:02,033 INFO][Epoch 21/50] Train Loss: 35.463298, Val Loss: 1449891.750000, RMSE: 5.646589, MAE: 4.167087, Accuracy: 0.6066, R2: 0.7079, Expl.Var: 0.7082
[2025-03-06 14:21:17,958 INFO][Epoch 31/50] Train Loss: 34.118202, Val Loss: 1378880.250000, RMSE: 5.506576, MAE: 4.082464, Accuracy: 0.6163, R2: 0.7222, Expl.Var: 0.7226
[2025-03-06 14:21:33,602 INFO][Epoch 41/50] Train Loss: 31.722393, Val Loss: 1255893.625000, RMSE: 5.255267, MAE: 3.927185, Accuracy: 0.6338, R2: 0.7471, Expl.Var: 0.7476
[2025-03-06 14:21:47,925 INFO][Epoch 50/50] Train Loss: 28.078746, Val Loss: 107

File data/W_est_shenzhen_pre_len1.npy found. Loading existing adjacency matrix estimated by GSL from training data.
GSL computed: Only GSL


[2025-03-06 14:21:49,776 INFO][Epoch 1/50] Train Loss: 90.770370, Val Loss: 838165.687500, RMSE: 4.293221, MAE: 3.054961, Accuracy: 0.7009, R2: 0.8319, Expl.Var: 0.8361
[2025-03-06 14:22:05,490 INFO][Epoch 11/50] Train Loss: 22.155054, Val Loss: 779778.812500, RMSE: 4.140989, MAE: 2.733996, Accuracy: 0.7115, R2: 0.8429, Expl.Var: 0.8429
[2025-03-06 14:22:21,162 INFO][Epoch 21/50] Train Loss: 21.780855, Val Loss: 770396.000000, RMSE: 4.116000, MAE: 2.724192, Accuracy: 0.7132, R2: 0.8447, Expl.Var: 0.8447
[2025-03-06 14:22:36,603 INFO][Epoch 31/50] Train Loss: 21.686204, Val Loss: 769832.250000, RMSE: 4.114494, MAE: 2.729474, Accuracy: 0.7133, R2: 0.8448, Expl.Var: 0.8449
[2025-03-06 14:22:52,245 INFO][Epoch 41/50] Train Loss: 21.667007, Val Loss: 776797.187500, RMSE: 4.133065, MAE: 2.809196, Accuracy: 0.7120, R2: 0.8437, Expl.Var: 0.8449
[2025-03-06 14:23:06,172 INFO][Epoch 50/50] Train Loss: 21.632922, Val Loss: 769739.312500, RMSE: 4.114245, MAE: 2.747931, Accuracy: 0.7133, R2: 0.8449

...

File data/W_est_losloop_pre_len4.npy found. Loading existing adjacency matrix estimated by GSL from training data.
GSL computed: Only GSL


[2025-03-06 14:42:04,589 INFO][Epoch 1/50] Train Loss: 6929.411973, Val Loss: 61636232.000000, RMSE: 19.588537, MAE: 14.194361, Accuracy: 0.6665, R2: -0.3686, Expl.Var: -0.5365
[2025-03-06 14:42:13,725 INFO][Epoch 11/50] Train Loss: 387.919924, Val Loss: 12234973.000000, RMSE: 8.727412, MAE: 5.769040, Accuracy: 0.8514, R2: 0.6048, Expl.Var: 0.6051
[2025-03-06 14:42:22,776 INFO][Epoch 21/50] Train Loss: 248.716331, Val Loss: 8405551.000000, RMSE: 7.233809, MAE: 4.468880, Accuracy: 0.8768, R2: 0.7287, Expl.Var: 0.7297
[2025-03-06 14:42:31,760 INFO][Epoch 31/50] Train Loss: 203.998822, Val Loss: 7065628.000000, RMSE: 6.632227, MAE: 4.090346, Accuracy: 0.8871, R2: 0.7717, Expl.Var: 0.7717
[2025-03-06 14:42:40,763 INFO][Epoch 41/50] Train Loss: 186.509908, Val Loss: 6552608.500000, RMSE: 6.386914, MAE: 3.811663, Accuracy: 0.8913, R2: 0.7883, Expl.Var: 0.7886
[2025-03-06 14:42:48,826 INFO][Epoch 50/50] Train Loss: 179.453769, Val Loss: 6288452.000000, RMSE: 6.256851, MAE: 3.804591, Accuracy:

File data/W_est_losloop_pre_len4.npy found. Loading existing adjacency matrix estimated by GSL from training data.
GSL computed: GSL+Adj


[2025-03-06 14:42:49,909 INFO][Epoch 1/50] Train Loss: 6786.746448, Val Loss: 48238924.000000, RMSE: 17.329370, MAE: 13.221812, Accuracy: 0.7050, R2: 0.1199, Expl.Var: 0.2125
[2025-03-06 14:42:58,936 INFO][Epoch 11/50] Train Loss: 525.462556, Val Loss: 17812280.000000, RMSE: 10.530373, MAE: 7.492685, Accuracy: 0.8207, R2: 0.4245, Expl.Var: 0.4246
[2025-03-06 14:43:07,783 INFO][Epoch 21/50] Train Loss: 375.161494, Val Loss: 12433444.000000, RMSE: 8.797914, MAE: 6.113583, Accuracy: 0.8502, R2: 0.5983, Expl.Var: 0.5984
[2025-03-06 14:43:16,910 INFO][Epoch 31/50] Train Loss: 328.410177, Val Loss: 10836886.000000, RMSE: 8.213651, MAE: 5.681320, Accuracy: 0.8602, R2: 0.6500, Expl.Var: 0.6504
[2025-03-06 14:43:26,080 INFO][Epoch 41/50] Train Loss: 298.577363, Val Loss: 9834196.000000, RMSE: 7.824442, MAE: 5.324487, Accuracy: 0.8668, R2: 0.6822, Expl.Var: 0.6824
[2025-03-06 14:43:34,146 INFO][Epoch 50/50] Train Loss: 278.112495, Val Loss: 9129491.000000, RMSE: 7.538888, MAE: 5.172353, Accuracy

CPU times: total: 20min 59s
Wall time: 23min 8s


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

model_cls_name = 'TGCN'
# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
datasets = ['shenzhen', 'losloop']

# Define a color palette for each method
colors = plt.cm.tab10.colors  # Using a predefined color palette (10 distinct colors)
method_colors = {
    "T-GCN": colors[0],           # T-GCN (Default)
    "T-GCN (GSL Only)": colors[1],  # T-GCN (GSL Only)
    "T-GCN (GSL + Adj)": colors[2], # T-GCN (GSL + Adj)
}

# Define line styles for each method
line_styles = {
    "T-GCN": "--",                # Dashed for T-GCN (Default)
    "T-GCN (GSL Only)": "-",       # Solid for T-GCN (GSL Only)
    "T-GCN (GSL + Adj)": ":",      # Dotted for T-GCN (GSL + Adj)
}

# Define markers for each method
markers = {
    "T-GCN": "o",                 # Circle for T-GCN (Default)
    "T-GCN (GSL Only)": "s",       # Square for T-GCN (GSL Only)
    "T-GCN (GSL + Adj)": "D",      # Diamond for T-GCN (GSL + Adj)
}

# Ensure the results directory exists
os.makedirs("results", exist_ok=True)

# Loop through each dataset
for dataset in datasets:
    # Loop through each measure
    for measure in measures:
        # Loop through each pre_len in pred_list
        for pre_len in pred_list:
            # Create a new figure for each pre_len and measure
            plt.figure(figsize=(12, 8))

            # Load the metrics for T-GCN (Default) for comparison
            metrics_file_tgcn = f"results/{model_cls_name}/metrics_{dataset}_seq12_pre{pre_len}_gsl0.csv"
            metrics_df_tgcn = pd.read_csv(metrics_file_tgcn)

            # Loop through each method
            for method, color in method_colors.items():
                # Map method names to their corresponding file suffixes
                if method == "T-GCN":
                    file_suffix = "gsl0"
                elif method == "T-GCN (GSL Only)":
                    file_suffix = "gsl1"
                elif method == "T-GCN (GSL + Adj)":
                    file_suffix = "gsl2"

                # Generate the file path dynamically based on pre_len and method
                metrics_file = f"results/{model_cls_name}/metrics_{dataset}_seq12_pre{pre_len}_{file_suffix}.csv"

                # Load the metrics CSV file
                metrics_df = pd.read_csv(metrics_file)

                # Plot the current measure for the current method
                plt.plot(
                    metrics_df["Epoch"],
                    metrics_df[measure],
                    linestyle=line_styles[method],
                    marker=markers[method],
                    color=color,
                    label=f"{method}", # (pre_len={pre_len})
                )

                # Highlight the first epoch where the current method is better than T-GCN (Default)
                if method != "T-GCN":  # Skip comparison for T-GCN (Default)
                    better_epoch = None
                    for epoch in range(len(metrics_df)):
                        tgcn_value = metrics_df_tgcn.loc[epoch, measure]
                        current_value = metrics_df.loc[epoch, measure]

                        # Check if the current method is better than T-GCN (Default)
                        if measure in ["RMSE", "MAE"]:
                            if current_value < tgcn_value:  # Lower is better
                                better_epoch = epoch + 1  # Epochs are 1-indexed
                                break
                        else:
                            if current_value > tgcn_value:  # Higher is better
                                better_epoch = epoch + 1
                                break

                    # Highlight the first epoch where the current method is better
                    if better_epoch is not None:
                        plt.axvline(
                            x=better_epoch,
                            color=color,
                            linestyle=":",
                            alpha=0.5,
                            # label=f"{method} better at epoch {better_epoch}", # (pre_len={pre_len})
                        )

            # Add labels, title, legend, and grid
            plt.xlabel("Epoch")
            plt.ylabel(measure)
            plt.title(f"{measure} over Epochs for {dataset} (pre_len={pre_len})")
            plt.legend()
            # plt.grid(True)

            # Save the figure as a PNG file
            filename = f"results/{model_cls_name}/{dataset}_{measure}_pre{pre_len}.png"
            plt.savefig(filename, bbox_inches="tight", dpi=300)
            plt.close()  # Close the figure to free up memory


In [4]:
import pandas as pd
import os

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
datasets = [ 'shenzhen', 'losloop']

# Ensure the results directory exists
os.makedirs("results", exist_ok=True)

# Loop through each dataset
for dataset in datasets:
    # Initialize a dictionary to store the results of the last epoch
    results = {}

    # Loop through each pre_len and measure to collect the last epoch results
    for pre_len in pred_list:
        # Generate the file paths dynamically based on pre_len and method
        metrics_file_gsl0 = f"results/{model_cls_name}/metrics_{dataset}_seq12_pre{pre_len}_gsl0.csv"
        metrics_file_gsl1 = f"results/{model_cls_name}/metrics_{dataset}_seq12_pre{pre_len}_gsl1.csv"
        metrics_file_gsl2 = f"results/{model_cls_name}/metrics_{dataset}_seq12_pre{pre_len}_gsl2.csv"

        # Load the metrics CSV files
        metrics_df_gsl0 = pd.read_csv(metrics_file_gsl0)
        metrics_df_gsl1 = pd.read_csv(metrics_file_gsl1)
        metrics_df_gsl2 = pd.read_csv(metrics_file_gsl2)

        # Get the last epoch results for all methods
        last_epoch_gsl0 = metrics_df_gsl0.iloc[-1]  # Last row for gsl0 (T-GCN)
        last_epoch_gsl1 = metrics_df_gsl1.iloc[-1]  # Last row for gsl1 (T-GCN GSL Only)
        last_epoch_gsl2 = metrics_df_gsl2.iloc[-1]  # Last row for gsl2 (T-GCN GSL + Adj)

        # Store the results in the dictionary with pre_len as part of the key
        results[f"T-GCN (pre_len={pre_len})"] = last_epoch_gsl0[measures]
        results[f"T-GCN (GSL Only) (pre_len={pre_len})"] = last_epoch_gsl1[measures]
        results[f"T-GCN (GSL + Adj) (pre_len={pre_len})"] = last_epoch_gsl2[measures]

    # Convert the results dictionary to a DataFrame
    results_df = pd.DataFrame(results).T

    # Add a column for pre_len to results_df
    results_df["pre\\_len"] = [pre_len for pre_len in pred_list for _ in range(3)]

    # Function to highlight the winner for each pre_len and measure
    def highlight_winner(df):
        highlighted_df = df.copy()
        for pre_len in pred_list:
            for measure in measures:
                # Get the values for the three methods for the current pre_len
                tgcn_value = df.loc[f"T-GCN (pre_len={pre_len})", measure]
                gsl_only_value = df.loc[f"T-GCN (GSL Only) (pre_len={pre_len})", measure]
                gsl_adj_value = df.loc[f"T-GCN (GSL + Adj) (pre_len={pre_len})", measure]

                # Determine the winner based on the measure
                if measure in ["RMSE", "MAE"]:
                    winner_value = min(tgcn_value, gsl_only_value, gsl_adj_value)  # Lower is better
                else:
                    winner_value = max(tgcn_value, gsl_only_value, gsl_adj_value)  # Higher is better

                # Highlight the winner
                if tgcn_value == winner_value:
                    highlighted_df.loc[f"T-GCN (pre_len={pre_len})", measure] = f"\\textbf{{{tgcn_value:.4f}}}"
                else:
                    highlighted_df.loc[f"T-GCN (pre_len={pre_len})", measure] = f"{tgcn_value:.4f}"

                if gsl_only_value == winner_value:
                    highlighted_df.loc[f"T-GCN (GSL Only) (pre_len={pre_len})", measure] = f"\\textbf{{{gsl_only_value:.4f}}}"
                else:
                    highlighted_df.loc[f"T-GCN (GSL Only) (pre_len={pre_len})", measure] = f"{gsl_only_value:.4f}"

                if gsl_adj_value == winner_value:
                    highlighted_df.loc[f"T-GCN (GSL + Adj) (pre_len={pre_len})", measure] = f"\\textbf{{{gsl_adj_value:.4f}}}"
                else:
                    highlighted_df.loc[f"T-GCN (GSL + Adj) (pre_len={pre_len})", measure] = f"{gsl_adj_value:.4f}"
        return highlighted_df

    # Apply the highlight function to the results DataFrame
    highlighted_results = highlight_winner(results_df)

    # Remove (pre_len={pre_len}) from method names
    highlighted_results.index = highlighted_results.index.str.replace(r" \(pre_len=\d+\)", "", regex=True)

    # Add a column for method names
    highlighted_results.insert(0, "Method", highlighted_results.index)

    # Reorder columns to make pre_len the first column and method-name the second column
    highlighted_results = highlighted_results[["pre\\_len", "Method"] + measures]

    # Generate the LaTeX table
    latex_table = highlighted_results.to_latex(
        escape=False,
        column_format="cl" + "c" * len(measures),  # Updated column format
        multicolumn_format="c",
        index=False  # Do not include the index in the LaTeX table
    )

    # Save the LaTeX table to a file
    table_filename = f"results/{model_cls_name}/{dataset}_results_table.tex"
    with open(table_filename, "w") as f:
        f.write(latex_table)

    # Print the LaTeX table
    print(f"LaTeX table for {dataset}:")
    print(latex_table)

LaTeX table for shenzhen:
\begin{tabular}{clcccc}
\toprule
 pre\_len &            Method &            RMSE &             MAE &        Accuracy &              R2 \\
\midrule
        1 &             T-GCN &          4.8657 &          3.6060 &          0.6610 &          0.7833 \\
        1 &  T-GCN (GSL Only) & \textbf{4.1142} & \textbf{2.7479} & \textbf{0.7133} & \textbf{0.8449} \\
        1 & T-GCN (GSL + Adj) &          4.8657 &          3.6060 &          0.6610 &          0.7833 \\
        2 &             T-GCN &          4.5064 &          3.2126 &          0.6860 &          0.8139 \\
        2 &  T-GCN (GSL Only) & \textbf{4.1618} & \textbf{2.8162} & \textbf{0.7100} & \textbf{0.8414} \\
        2 & T-GCN (GSL + Adj) &          4.5064 &          3.2126 &          0.6860 &          0.8139 \\
        3 &             T-GCN &          4.6847 &          3.4161 &          0.6735 &          0.7989 \\
        3 &  T-GCN (GSL Only) & \textbf{4.1928} & \textbf{2.8060} & \textbf{0.7078} & \textb

In [5]:
import os

# Define the prediction lengths and measures
pred_list = [1, 2, 3, 4]
measures = ["RMSE", "MAE", "Accuracy", "R2"]
datasets = [ 'shenzhen', 'losloop']

# Ensure the results directory exists
os.makedirs("results", exist_ok=True)

# Function to generate LaTeX table for a dataset
def generate_latex_table(dataset):
    latex_code = f"""
\\begin{{table}}[htbp]
\\centering
\\caption{{Results for {dataset}: RMSE, MAE, Accuracy, and R2 for pre\\_len=1, 2, 3, 4.}}
\\begin{{tabular}}{{|c|{'c' * len(measures)}|}}
\\hline
pre\\_len & {' & '.join(measures)} \\\\
\\hline
"""

    for pre_len in pred_list:
        row = f"{pre_len} "
        for measure in measures:
            image_path = f"{dataset}_{measure}_pre{pre_len}.png"
            row += f"& \\includegraphics[width=0.2\\textwidth]{{{image_path}}} "
        row += "\\\\ \\hline"
        latex_code += row + "\n"

    latex_code += """
\\end{tabular}
\\end{table}
"""
    return latex_code

# Generate LaTeX tables for each dataset
for dataset in datasets:
    latex_table = generate_latex_table(dataset)
    print(f"LaTeX table for {dataset}:")
    print(latex_table)

    # Save the LaTeX table to a file
    table_filename = f"results/{model_cls_name}_{dataset}_images_table.tex"
    with open(table_filename, "w") as f:
        f.write(latex_table)

LaTeX table for shenzhen:

\begin{table}[htbp]
\centering
\caption{Results for shenzhen: RMSE, MAE, Accuracy, and R2 for pre\_len=1, 2, 3, 4.}
\begin{tabular}{|c|cccc|}
\hline
pre\_len & RMSE & MAE & Accuracy & R2 \\
\hline
1 & \includegraphics[width=0.2\textwidth]{shenzhen_RMSE_pre1.png} & \includegraphics[width=0.2\textwidth]{shenzhen_MAE_pre1.png} & \includegraphics[width=0.2\textwidth]{shenzhen_Accuracy_pre1.png} & \includegraphics[width=0.2\textwidth]{shenzhen_R2_pre1.png} \\ \hline
2 & \includegraphics[width=0.2\textwidth]{shenzhen_RMSE_pre2.png} & \includegraphics[width=0.2\textwidth]{shenzhen_MAE_pre2.png} & \includegraphics[width=0.2\textwidth]{shenzhen_Accuracy_pre2.png} & \includegraphics[width=0.2\textwidth]{shenzhen_R2_pre2.png} \\ \hline
3 & \includegraphics[width=0.2\textwidth]{shenzhen_RMSE_pre3.png} & \includegraphics[width=0.2\textwidth]{shenzhen_MAE_pre3.png} & \includegraphics[width=0.2\textwidth]{shenzhen_Accuracy_pre3.png} & \includegraphics[width=0.2\textwidth]{s

In [6]:
if 'google.colab' in str(get_ipython()): 
    from google.colab import files
    # Zip the results folder
    shutil.make_archive('results', 'zip', 'results')
    # Download the zipped file
    files.download('results.zip')